In [4]:
import pandas as pd
import datetime
import requests
import urllib
import json
from urllib.parse import urlencode
from datetime import date
import calendar
import numpy as np
import seaborn as sns
import math as m
import matplotlib.pyplot as plt

In [5]:
def yandex_file_download(public_key):
    base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?'
    public_key = public_key
    # Получаем загрузочную ссылку
    final_url = base_url + urlencode(dict(public_key=public_key))
    response = requests.get(final_url)
    download_url = response.json()['href']
    h = json.loads(response.text)['href'] # 'парсинг' ссылки на скачивание
    
    return h

In [ ]:
olist_customers = pd.read_csv(yandex_file_download('https://disk.yandex.ru/d/FUi5uSd6BfG_ig'), 
                              sep=',', error_bad_lines=False, comment='#' , encoding='cp1251')

In [ ]:
 olist_orders = pd.read_csv(yandex_file_download('https://disk.yandex.ru/d/t9Li4JOfgxuUrg'), 
                              sep=',', error_bad_lines=False, comment='#' , encoding='cp1251')

Уникальные статусы заказов в таблице olist_orders_dataset:

- created —  создан
- approved —  подтверждён
- invoiced —  выставлен счёт
- processing —  в процессе сборки заказа
- shipped —  отгружен со склада
- delivered —  доставлен пользователю
- unavailable —  недоступен
- canceled —  отменён


In [ ]:
olist_items = pd.read_csv(yandex_file_download('https://disk.yandex.ru/d/Gbt-yAcQrOe3Pw'), 
                              sep=',', error_bad_lines=False, comment='#' , encoding='cp1251')

In [ ]:
olist_customers.isna().sum()

- customer_id — позаказный идентификатор пользователя

- customer_unique_id —  уникальный идентификатор пользователя  (аналог номера паспорта)

- customer_zip_code_prefix —  почтовый индекс пользователя

- customer_city —  город доставки пользователя

- customer_state —  штат доставки пользователя

In [ ]:
olist_customers.info()

In [ ]:
olist_orders.isna().sum()

- order_id —  уникальный идентификатор заказа (номер чека)

- customer_id —  позаказный идентификатор пользователя

- order_status —  статус заказа

- order_purchase_timestamp —  время создания заказа

- order_approved_at —  время подтверждения оплаты заказа

- order_delivered_carrier_date —  время передачи заказа в логистическую службу

- order_delivered_customer_date —  время доставки заказа

- order_estimated_delivery_date —  обещанная дата доставки

In [ ]:
olist_orders.info()

In [ ]:
olist_items.isna().sum()

- rder_id —  уникальный идентификатор заказа (номер чека)

- order_item_id —  идентификатор товара внутри одного заказа

- product_id —  ид товара (аналог штрихкода)

- seller_id — ид производителя товара

- shipping_limit_date —  максимальная дата доставки продавцом для передачи заказа партнеру по логистике

- price —  цена за единицу товара

- freight_value —  вес товара

In [ ]:
olist_items.info()

1. Сколько у нас пользователей, которые совершили покупку только один раз?

- Объединяем таблицы olist_customers и olist_orders, по customer_id
- Статус заказа все кроме created 
- Группируем пользователей и считаем кол-во заказов 
- Отбираем пользователей с 1 заказом , считаем их

In [ ]:
ferst_data = olist_orders[['customer_id', 'order_id','order_status']].merge(olist_customers[['customer_id','customer_unique_id']], how = 'left', on = 'customer_id')

In [ ]:
ferst_data[~((ferst_data.order_status == 'created'))].groupby('customer_unique_id', as_index = False).agg({'order_id':'count'})\
.rename(columns = {'order_id':'order_amount'}).query('order_amount == 1').shape

2. Сколько заказов в месяц в среднем не доставляется по разным причинам (вывести детализацию по причинам)?

- Берем таблицу olist_orders
- Вычленяем недоставленные заказы (unavailable, canceled) 
- Групперуем по статусу и месяцам 
- Счетаем сколько заказов недоставленно в каждый из месяцев 
- Берем среднее помесяцам

In [ ]:
sec_data = olist_orders.query('order_status == "unavailable" or order_status == "canceled"')[olist_orders.order_delivered_customer_date.isna()==True]

In [ ]:
olist_orders.query('order_status == "unavailable" or order_status == "canceled"').shape

Есть 6 заказов у которых есть дата доставки , но статус = недействителен/отменен

In [ ]:
sec_data['order_estimated_delivery_date'] = pd.to_datetime(sec_data.order_estimated_delivery_date)
# Переводим столбец ореинтировочной даты доставки в формат даты 

In [ ]:
sec_data.groupby(['order_status', pd.Grouper(key='order_estimated_delivery_date', freq='M')]).order_id.count().to_frame()\
.reset_index()\
.groupby('order_status').agg({'order_id':'mean'})
#pd.Grouper сразу сам группирует по месяцам (freq = M), очень удобно для работы с таймлайнами 

3. По каждому товару определить, в какой день недели товар чаще всего покупается.

- Подходят статусы все кроме created
- Мерджим таблицу заказов и таблицу товаров
- Создаемм столбец с днем недели покупки для каждого товара
- Агрегируем по каждому товару и дню недели, выбираем максимальное значение пары : товар/день недели 


In [ ]:
therd_task = olist_items.merge(olist_orders.query('order_status != "created"')[['order_id', 'order_purchase_timestamp']], how= 'left', on = 'order_id')

In [ ]:
therd_task['order_purchase_timestamp'] = pd.to_datetime(therd_task.order_purchase_timestamp)

In [ ]:
therd_task['weekday'] = therd_task['order_purchase_timestamp'].dt.day_name()

In [ ]:
therd_task.groupby(['weekday','product_id'], as_index = False).agg({'order_id':'count'})\
.pivot_table(columns = 'product_id', index = 'weekday', values = 'order_id').idxmax()\
.to_frame().rename(columns = {0: 'top_day'}).reset_index()
#idxmax находит первое максимальное вхождение по каждому столбцу(товару)и переварачивает таблицу

 4.Сколько у каждого из пользователей в среднем покупок в неделю (по месяцам)? Не стоит забывать, что внутри месяца может быть не целое количество недель. Например, в ноябре 2021 года 4,28 недели. И внутри метрики это нужно учесть.

1. Для каждого месяца в датасете рассчитать количество дней
2. Выделить количество недель внутри месяца, поделив кол-во дней на 7
3. Отфильтровать заказы в статусах, несоответствующих покупке
4. Приджойнить к orders уникальные идентификаторы пользователей
5. Добавить к orders колонку с месяцем покупки
6. Для каждого пользователя внутри месяца рассчитать общее количество заказов
7. К orders приджойнить количества неделя по месяцам
8. Поделить кол-во заказов по пользователям-месяцам на количество кол-во недель в месяце

In [ ]:
forth_task = olist_orders[['customer_id', 'order_id','order_status','order_purchase_timestamp']].merge(olist_customers[['customer_id','customer_unique_id']], how = 'left', on = 'customer_id')\
.query('order_status != "created"')

In [ ]:
forth_task.shape

In [ ]:
forth_task['order_purchase_timestamp'] = pd.to_datetime(forth_task.order_purchase_timestamp)

In [ ]:
forth_task['week_in_month'] = forth_task.order_purchase_timestamp.dt.daysinmonth/7

In [ ]:
forth_task['month'] = forth_task.order_purchase_timestamp.dt.strftime('%m/%Y')

In [ ]:
x = forth_task.groupby(['customer_unique_id', 'month'], as_index = False).agg({'customer_id':'count'})

In [ ]:
x = x.merge(forth_task[['week_in_month', 'month']].drop_duplicates(), on = 'month', how = 'left' )

In [ ]:
x = x.rename(columns = {'customer_id':'sails'})

In [ ]:
x['by_per_week_in_month'] = x.sails/x.week_in_month

5. Используя pandas, проведи когортный анализ пользователей. В период с января по декабрь выяви когорту с самым высоким retention на 3й месяц. 

- Для каждого пользователя выбрать первую дату доставки
- Определить когорты, путем разности даты первой покупки и даты покупки
- Выделлить целое кол-во месяце( когорты определяем месячные )
- Посчитать ретеншен по когортам
- Визуализировать хитмапом

In [ ]:
task5 = olist_orders[~olist_orders.order_status.isin(['created'])].merge(
            olist_customers[['customer_id', 'customer_unique_id']], 
            on='customer_id', 
            how='left')

In [ ]:
task5['order_purchase_timestamp'] = pd.to_datetime(task5.order_purchase_timestamp)

In [ ]:
task5 = task5.merge(task5.groupby('customer_unique_id', as_index = False).agg({'order_purchase_timestamp':'min'}).rename(columns = {'order_purchase_timestamp':'ferst_buy'}),on = 'customer_unique_id', how = 'left')

In [ ]:
task5 = task5[['order_id', 'customer_unique_id', 'order_purchase_timestamp', 'ferst_buy', 'order_status']]

In [ ]:
task5['month'] = task5.ferst_buy.dt.strftime('%m/%Y')

In [ ]:
task5['days_dist'] = (task5.order_purchase_timestamp - task5.ferst_buy).dt.days 

In [ ]:
task5['days_in_month'] = forth_task.order_purchase_timestamp.dt.daysinmonth

In [ ]:
from math import trunc

In [ ]:
task5['chort'] = (task5.days_dist / task5.days_in_month).fillna(0).apply(lambda x: m.trunc(x))
# Выделяем целое кол-во месяцев у каждой покупки, чтобы было удобнее когорты выделять 

In [ ]:
fine_5 = task5.pivot_table(index ='month', columns = 'chort', values = 'order_id', aggfunc= 'count' )

In [ ]:
all_users = fine_5[0]

In [ ]:
#Наводим красоту (сортировка)
retention = fine_5.divide(all_users, axis=0).round(3) #Формируем когорты
retention = retention.reset_index()
retention['month'] = pd.to_datetime(retention['month']).dt.date
retention = retention.sort_values('month').set_index('month')

In [ ]:
plt.figure(figsize=(18,14))
plt.title('Retention')
ax = sns.heatmap(data=retention, annot=True, vmin=0.0, vmax=0.005, cmap='YlGnBu')
ax.set_yticklabels(retention.index)
fig=ax.get_figure()
plt.show()

6. Часто для качественного анализа аудитории использую подходы, основанные на сегментации. Используя python, построй RFM-сегментацию пользователей, чтобы качественно оценить свою аудиторию. В кластеризации можешь выбрать следующие метрики: R - время от последней покупки пользователя до текущей даты, F - суммарное количество покупок у пользователя за всё время, M - сумма покупок за всё время. Подробно опиши, как ты создавал кластеры. Для каждого RFM-сегмента построй границы метрик recency, frequency и monetary для интерпретации этих кластеров. Пример такого описания: RFM-сегмент 132 (recency=1, frequency=3, monetary=2) имеет границы метрик recency от 130 до 500 дней, frequency от 2 до 5 заказов в неделю, monetary от 1780 до 3560 рублей в неделю.

In [ ]:
from numpy.lib.function_base import quantile


In [ ]:
ferst_part = olist_orders[['order_status','customer_id', 'order_id', 'order_purchase_timestamp']]\
.query('order_status != ["created"] & order_status != ["unavailable"] & order_status != ["canseled"]')\
.merge(olist_customers[['customer_unique_id', 'customer_id']], how = 'left', on = 'customer_id')

In [ ]:
fool_data = olist_items[['order_id', 'order_item_id', 'product_id', 'shipping_limit_date', 'price']].merge(ferst_part, how = 'left', on = 'order_id')

In [ ]:
olist_items.shape

In [ ]:
fool_data.shape

In [ ]:
fool_data.info()

In [ ]:
fool_data['shipping_limit_date'] = pd.to_datetime(fool_data.shipping_limit_date)

In [ ]:
fool_data['order_purchase_timestamp'] = pd.to_datetime(fool_data.shipping_limit_date)

In [ ]:
fool_data.order_id.value_counts()


In [ ]:
las_date = fool_data.order_purchase_timestamp.max()

Опредлились с Днями прошедшими с последней покупки

In [ ]:
RFM_data = fool_data.groupby('customer_unique_id').agg({'order_purchase_timestamp':'max', 'price':'sum', 'order_id':'nunique'}).reset_index()\
.rename(columns = {'order_purchase_timestamp':'last_buy', 'order_id':'orders_total', 'price' : 'total_cash'})

In [ ]:
RFM_data['days_from_last_buy'] = (las_date - RFM_data.last_buy).dt.days

In [ ]:
RFM_data.head()

In [ ]:
RFM_data.orders_total.describe()

In [ ]:
quantils = RFM_data.quantile(q=[0.25, 0.5, 0.75])

Так как заказы распределены совсем не нормально , то для их анализа используем количественные показатели

In [ ]:
def RClass(value,parameter_name,quantiles_table):
    if value <= quantiles_table[parameter_name][0.25]:
        return 1
    elif value <= quantiles_table[parameter_name][0.50]:
        return 2
    elif value <= quantiles_table[parameter_name][0.75]: 
        return 3
    else:
        return 4
def FClass(value, parameter_name,quantiles_table):
    if value <= quantiles_table[parameter_name][0.25]:
        return 4
    elif value <= 4:
        return 3
    elif value <= 8: 
        return 2
    else:
        return 1


def MClass(value, parameter_name,quantiles_table):
    if value <= quantiles_table[parameter_name][0.25]:
        return 4
    elif value <= quantiles_table[parameter_name][0.50]:
        return 3
    elif value <= quantiles_table[parameter_name][0.75]: 
        return 2
    else:
        return 1

In [ ]:
rfmSegmentation = RFM_data

In [ ]:
rfmSegmentation['R_Quartile'] = rfmSegmentation['days_from_last_buy'].apply(RClass, args=('days_from_last_buy',quantils))

rfmSegmentation['F_Quartile'] = rfmSegmentation['orders_total'].apply(FClass, args=('orders_total',quantils))

rfmSegmentation['M_Quartile'] = rfmSegmentation['total_cash'].apply(MClass, args=('total_cash',quantils))

rfmSegmentation['RFMClass'] = rfmSegmentation.R_Quartile.map(str)+ rfmSegmentation.F_Quartile.map(str)    + rfmSegmentation.M_Quartile.map(str)

In [ ]:
rfmSegmentation.groupby('RFMClass').agg({'customer_unique_id':'count', 'total_cash':'sum', 'days_from_last_buy':'mean', 'orders_total':'mean'})\
.rename(columns = {'customer_unique_id':'users_total', 'days_from_last_buy':'mean_days_from_last_buy'  })\
.reset_index()